# Preliminaries
Write requirements to file, anytime you run it, in case you have to go back and recover dependencies.

Requirements are hosted for each notebook in the companion github repo, and can be pulled down and installed here if needed. Companion github repo is located at https://github.com/azunre/transfer-learning-for-nlp

In [2]:
!pip freeze > kaggle_image_requirements.txt

# Open Ended Text Generation with GPT-2

In [3]:
# Pipeline uses `gpt2` by default, but we specify it explicitly to be fully transparent
from transformers import pipeline
gpt = pipeline('text-generation',model='gpt2')

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


Now, let's generate some text with GPT-2

In [4]:
gpt("Transfer learning is a field of study", max_length=100)

[{'generated_text': "Transfer learning is a field of study that has been around for centuries, and one that requires a thorough grounding in mathematics in order to understand the complexities of these systems. If you go to the library for your high school physics course, you know you're on the right track. The only problem with this position is that people don't ask questions. The only thing they really do ask is: how do we figure out how to apply these processes to the rest of physics and other sciences?\n\nIn"}]

A nonexhaustive list of other model choices suitable for text generation within the transformers library include "ctrl" (CTRL - huge! too big for Kaggle), "xlnet-base-cased" (XLNet), "transfo-xl-wt103" (Transformer XL)... These often need to be padded very carefully to work well, GPT-2 is the safest choice for open-ended text generation. See https://huggingface.co/transformers/usage.html#text-generation for more.

# Conversational Text Generation with DialoGPT

DialoGPT is an extension of GPT to conversational response generation

In [5]:
from transformers import AutoModelWithLMHead, AutoTokenizer # you can use these utility classes that automatically load the right classes
from transformers import GPT2LMHeadModel, GPT2Tokenizer # or these more specific classes directly
import torch

tokenizer = GPT2Tokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = GPT2LMHeadModel.from_pretrained("microsoft/DialoGPT-medium")

In [6]:
# Chat for 5 Lines
conversation_length = 5
for step in range(conversation_length):
    # encode new user input, add end-of-sentence token, return tensor
    new_user_inputs_ids = tokenizer.encode(input("User: ") + tokenizer.eos_token, return_tensors='pt')
    
    # add new input to chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_inputs_ids], dim=1) if step > 0 else new_user_inputs_ids
    
    # generate a response of up to max_length tokens
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    
    # display response
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

User: hi robot
DialogGPT: Hello, human.
User: huh?
DialogGPT: I'm a bot.
User: ok, what is your name?
DialogGPT: Robot. I'm a robot.
User: Alright then
DialogGPT: Robot. I'm a robot.
User: Say something else
DialogGPT: Robot. I'm a robot.
